In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 
#list of input files

def get_mur():
    file_location = 's3://mur-sst/zarr'
    ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
    masked = ds_sst.where(((ds_sst.mask==1) or (ds_sst.sea_ice_fraction<.15)))
    masked = masked.drop({'analysis_error','mask','sea_ice_fraction'})
#    masked = ds_sst.where(((ds_sst.mask==1)))
#    masked.analysed_sst[0,2000:5000:20,18000:19000:20].plot()
#    masked = ds_sst.where(((ds_sst.mask==1) or (ds_sst.sea_ice_fraction<.15)))
#    masked.analysed_sst[0,1000:6000:200,18000:18100:10].plot()
    return masked

def get_cmc():
    file_location = 'F:/data/sat_data/sst/cmc/zarr'
    ds = xr.open_zarr(file_location)
    masked = ds.where((ds.mask<=1) & (ds.sea_ice_fraction<.15))
    masked = masked.drop({'analysis_error','mask','sea_ice_fraction'})
    masked['analysed_sst']-=273.15
    return masked

In [ ]:
masked = get_cmc()
masked = masked.sel(time=slice('1999-01-01','2019-12-31'))
#sst_climatology = masked.groupby('time.month').mean(dim='time',keep_attrs=True,skipna=True)
sst_climatology_yr = masked.mean(dim='time',keep_attrs=True,skipna=True)
sst_climatology_yr

In [ ]:
df = pd.read_excel('./../data/sst_error.xlsx')
ds = df.to_xarray().rename({'SST':'sst'})
ds = ds.swap_dims({'index':'sst'}).drop('index')
ds['std_dif']=ds.std_11GHz-ds.std_7GHz

In [ ]:
ds.std_11GHz.plot()
ds.std_7GHz.plot()

In [ ]:
#from dask_gateway import Gateway
#from dask.distributed import Client
#gateway = Gateway()
##cluster = gateway.new_cluster()
#cluster.adapt(minimum=1, maximum=20)
#client = Client(cluster)
#cluster

In [ ]:
ds_sst = sst_climatology_yr.load()

In [ ]:
err_map = xr.DataArray(ds_sst.analysed_sst.data*np.NaN,coords={'lat':ds_sst.lat,'lon':ds_sst.lon},dims=('lat','lon'))
err_map = xr.Dataset({'err':err_map})
isst = (ds_sst.analysed_sst*10).astype('int')
for i in range(-30,360):
    err = ds.interp(sst=i/10,method='linear')
    err = err.std_dif.data
    tem = xr.where(isst==i,err,err_map)
    err_map['err'] = tem.err

In [ ]:
tem.err.plot()

In [ ]:
import cartopy.crs as ccrs
plt.figure(figsize=(20,8))
ax = plt.axes(projection=ccrs.PlateCarree())
cs1 = plt.pcolormesh(tem.lon,tem.lat,tem.err,vmin=0,vmax=.3,cmap='magma')
ax.coastlines()
cb = plt.colorbar()
cb.set_label(label='Increase in STD (K)',fontsize=16,)
plt.savefig('C:/Users/gentemann/Google Drive/f_drive/docs/proposals/nasa/EVM/7_11_err.png')